# Import Dependencies

In [ ]:
#data tools
import pandas as pd
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
#from pymatgen.vis import structure_chemview as viz

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

### User Create ~/.mpkey.txt if it doesn't already exist

In [ ]:
#key security
try:
    user = str(input('Paste MP API key: '))
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user is None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

## User Prompted To pick their SemiConductor of Choice
### Choice (a): query MP and fliter on properties. Dataframe is updated with each selection.

In [ ]:
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

In [ ]:
#if you don't, search for your structure by filtering


In [ ]:
# if you know the mp-id: give it here to get a structure object
mpid = widgets.IntText(
        value=2133,
        description='MPID:',
        disabled=False
)

display(mpid)

In [ ]:
struct = rester.get_structure_by_material_id("mp-"+str(mpid.value), final = False, conventional_unit_cell=True)

In [ ]:
#debugging struct
type(struct.sites[0].a)

In [ ]:
# if you're just hunting for something we display the structure for you.
mv = viz.quick_view(struct)
mv.ball_and_stick()

In [ ]:
# This is passed to the simtool to perform simulations
# structure objects obtained directly from remote database queries must be passed to simtool as dictionaries to preserve maximum numerical precision
# quantum espresso is very sensitive to precision especially for relaxation calculations.
struct_dict = struct.as_dict()#to(fmt="JSON")
struct_dict

In [ ]:
#debug poscar conversion
from pymatgen.core import Structure
with open("./POSCAR", "w") as f:
    POSCAR = f.write(POSCAR_str)
    f.close()

struct2 = Structure.from_file("./POSCAR")

In [ ]:
struct2.sites[0].a

### Choice (b): upload your own poscar directly. No query necessary.

In [ ]:
#users can pass their own structure as a poscar. the simtool will take the file contents as a dictionary which is produced here
struct = Structure.from_file("./POSCAR")
struct_dict = struct.as_dict()
struct_dict

# Perform Structure Relaxation and SCF and Phonon computation and spectra extraction using simtool

### Find 670raman simtool notebook and confirm

In [ ]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

In [ ]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

### User Set Validated Inputs
670raman will automatically activate your rest api interface to Materials Project
if you have the dotfile ".mpkey.txt" in your home directory.

Otherwise, it will attempt to generate a realistic crystal structure from your chemical discription

In [ ]:
#Enter your values with units! The simtool will make sure you know what you're talking about.
inputs = getSimToolInputs(simToolLocation)

In [ ]:
inputs

In [ ]:
# skip this: reimplementing widget definitions wip
class ToolSet():
    def __init__(self, inputs, struct_dict): #users_pp_file_list
        self.pp_list = []
        for filename in os.listdir("./simtool/pseudo/"):
            f = os.path.join("./simtool/pseudo/", filename)
            # get a list of all the PPs -- is this best instatiated here or globally?
            # if instanced here, the user could probably pass their own PPs to the constructor as well
            if os.path.isfile(f):
                self.pp_list.append(filename)

        # TODO: filter by selected compound compositions
        self.filtered_pp_list = self.pp_list

    def _create_widgets(self):
        self.log = widgets.Select(
            options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
            value='DEBUG',
            # rows=10,
            description='Log Level:',
            disabled=False
        )
        self.walltime = widgets.Text(
            value='01:00:00',
            placeholder='walltime',
            description='walltime:',
            disabled=False
        )
        self.numnodes = widgets.IntText(
            value=8,
            placeholder='nodes',
            description='nodes:',
            disabled=False
        ) 
        self.button = widgets.Button(
            description='run simtool',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='run to submit qe simtool'
        )
        self.button.on_click(self._on_button_clicked)
        self.ecutwfc = widgets.BoundedFloatText(
            value=50,
            min=50,
            max=400,
            step=10,
            description='ecutwfc:',
            disabled=False
        )
        self.ecutrho = widgets.BoundedFloatText(
            value=200,
            min=200,
            max=1600,
            step=40,
            description='ecutrho:',
            disabled=False
        )
        self.smearing = widgets.Select(
            options=['smearing','fixed'],
            value='fixed',
            rows = 2,
            description='smearing:',
            disabled=False
        )
        self.pp_menu1 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 1:',
            disabled=False
        )
        self.pp_menu2 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 2:',
            disabled=False
        )
        self.output = widgets.Output()
        def _on_button_clicked(self, change):
            with output:
                print("submitting sim2l run with formula" , self.compound.value, self.spacegroup.value)
                runSim2l()
                r = Run(simToolLocation,inputs)

        # display(c, s, button, output)
        def runSim2l():
            inputs['loglevel'].value = log.value
            inputs['walltime'].value = walltime.value
            inputs['numnodes'].value = numnodes.value
            inputs['ecutwfc'].value = ecutwfc.value
            inputs['ecutrho'].value = ecutrho.value
            inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
            inputs['smearing'].value = smearing.value
            inputs['struct_dict'].value = struct_dict

In [ ]:
log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',

compound = widgets.Text(
    value='ZnO',
    placeholder='chemical formula',
    description='Compound:',
    disabled=False
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudopotential 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudopotential 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        print("submitting sim2l run with formula" , compound.value)
        runSim2l()
        r = Run(simToolLocation,inputs)
        r.getResultSummary()
        print(r.read('spectra'))
        
        
button.on_click(on_button_clicked)

structure = widgets.VBox([compound,spacegroup])
simulation = widgets.VBox([ecutrho, ecutwfc, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)

In [ ]:
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [ ]:
print(r.read('logreport'))

In [ ]:
r.read('spectra')

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)